# Train the model

In [ ]:
from ultralytics import YOLO
import os
import boto3
import re

In [3]:
! tar -xzf dataset-full.tar.gz

In [8]:
EPOCHS=int(os.environ.get("epochs", 50))
IMGSZ=640
NB_FROZEN_LAYER=0
DATA_CONFIG_PATH = "utils/data.yaml"

BASE_MODEL_BUCKET = "model-registry"
MODEL_PATH = "models/model.onnx"

AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
AWS_S3_BUCKET = os.environ.get("AWS_S3_BUCKET")
AWS_S3_ENDPOINT = os.environ.get("AWS_S3_ENDPOINT")

In [5]:
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)
client = session.client("s3", endpoint_url=AWS_S3_ENDPOINT)

In [6]:
client.download_file(BASE_MODEL_BUCKET, "models/default/best.pt", "base-model.pt")

## Load the base model and tune the model

In [7]:
model = YOLO('base-model.pt')
results = model.train(data=DATA_CONFIG_PATH, epochs=EPOCHS, imgsz=IMGSZ, freeze=NB_FROZEN_LAYER)

Ultralytics YOLOv8.2.32 🚀 Python-3.9.16 torch-2.3.1+cu121 CPU (AMD EPYC 7763 64-Core Processor)
engine/trainer: task=detect, mode=train, model=base-model.pt, data=utils/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=22, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

[W NNPACK.cpp:61] Could not initialize NNPACK! Reason: Unsupported hardware.


Freezing layer 'model.6.m.0.cv2.conv.weight'
Freezing layer 'model.6.m.0.cv2.bn.weight'
Freezing layer 'model.6.m.0.cv2.bn.bias'
Freezing layer 'model.6.m.1.cv1.conv.weight'
Freezing layer 'model.6.m.1.cv1.bn.weight'
Freezing layer 'model.6.m.1.cv1.bn.bias'
Freezing layer 'model.6.m.1.cv2.conv.weight'
Freezing layer 'model.6.m.1.cv2.bn.weight'
Freezing layer 'model.6.m.1.cv2.bn.bias'
Freezing layer 'model.7.conv.weight'
Freezing layer 'model.7.bn.weight'
Freezing layer 'model.7.bn.bias'
Freezing layer 'model.8.cv1.conv.weight'
Freezing layer 'model.8.cv1.bn.weight'
Freezing layer 'model.8.cv1.bn.bias'
Freezing layer 'model.8.cv2.conv.weight'
Freezing layer 'model.8.cv2.bn.weight'
Freezing layer 'model.8.cv2.bn.bias'
Freezing layer 'model.8.m.0.cv1.conv.weight'
Freezing layer 'model.8.m.0.cv1.bn.weight'
Freezing layer 'model.8.m.0.cv1.bn.bias'
Freezing layer 'model.8.m.0.cv2.conv.weight'
Freezing layer 'model.8.m.0.cv2.bn.weight'
Freezing layer 'model.8.m.0.cv2.bn.bias'
Freezing layer '

train: Scanning /opt/app-root/src/activity-notebooks/dataset/labels/train.cache... 74 images, 0 backgrounds, 0 corrupt: 100%|██████████| 74/74 [00:00<?, ?it/s]
val: Scanning /opt/app-root/src/activity-notebooks/dataset/labels/val.cache... 10 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10/10 [00:00<?, ?it/s]


Plotting labels to runs/detect/train7/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train7
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      26/50         0G      1.834      1.285      1.221         32        640: 100%|██████████| 5/5 [00:21<00:00,  4.20s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.42s/it]

                   all         10         30      0.856       0.85      0.826       0.45



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      27/50         0G       1.82      1.144      1.177         65        640:  40%|████      | 2/5 [00:11<00:17,  5.72s/it]

KeyboardInterrupt



In [ ]:
model = YOLO('runs/detect/train/weights/best.pt')

In [ ]:
model.export(format="onnx")

In [ ]:
model_path = "runs/detect/train/weights/best.onnx"

In [ ]:
client.upload_file(model_path, AWS_S3_BUCKET, MODEL_PATH)

In [ ]:
client.upload_file('runs/detect/train/weights/best.pt', AWS_S3_BUCKET, 'models/new-model.pt')

In [ ]:
client.upload_file('runs/detect/train/results.csv', AWS_S3_BUCKET, 'results.csv')